<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/prep_data/hurr_argopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install argopy

In [ ]:
# https://nbviewer.org/github/euroargodev/argopy/blob/master/docs/tutorials/basic_features_core_01.ipynb
import argopy

import pandas as pd
import numpy as np
import xarray as xr
from datetime import timedelta

import matplotlib.pyplot as plt

In [ ]:
year  = 2024
basin = 'north_atlantic'
storm = 'MILTON' # 'BERYL' 'HELENE' 'KIRK' 'MILTON'

dPath = f'drive/MyDrive/datasets/hurr/{str(year)}/SAT_datasets/'
dT = 1 # ? days before/after storm

In [ ]:
argopy.set_options(src='gdac', gdac='https://data-argo.ifremer.fr/')

## Read hurricane processed track info

In [ ]:
# track
fName_track = dPath + f'{year}_{basin}_{storm}.nc'
track = xr.open_dataset(fName_track)

track_lon_min, track_lon_max = [track.lon.values.min(), track.lon.values.max()]
track_lat_min, track_lat_max = [track.lat.values.min(), track.lat.values.max()]

In [ ]:
# Get the initial and final dates of storm
initial_date = pd.to_datetime(track.time[0].values)
final_date = pd.to_datetime(track.time[-1].values)

one_day_before = initial_date - timedelta(days=dT)
one_day_after = final_date + timedelta(days=dT)

str_one_day_before = one_day_before.strftime('%Y-%m-%d')
str_one_day_after = one_day_after.strftime('%Y-%m-%d')

print(f"{storm} track\nLon: [{track_lon_min}, {track_lon_max}]\tLat: [{track_lat_min}, {track_lat_max}]")

print(f"Plus/minus {dT}-days:\t{str_one_day_before}-- {str_one_day_after}")

In [ ]:
BOX = [track_lon_min, track_lon_max, track_lat_min, track_lat_max,\
       0, 2000,\
       str_one_day_before, str_one_day_after]

f = argopy.DataFetcher(src='gdac', progress=True)
f = f.region(BOX)  # Define a data selection
f = f.load()       # Load data

In [ ]:
ds=f.data

In [ ]:
print(ds)

In [ ]:
# Remove the 'raw_attrs' attribute as it is a dictionary and cannot be saved to netCDF
if 'raw_attrs' in ds.attrs:
    del ds.attrs['raw_attrs']

ds.to_netcdf(dPath + f'{year}_{basin}_{storm}_argo.nc')

In [ ]:
fig = plt.figure(figsize=(8, 12))
ax1 = fig.add_subplot(1, 1, 1)
ax2 = ax1.twiny()

ax1.plot(ds.TEMP.values, -ds.PRES.values, 'b.', label='Temperature')
ax2.plot(ds.PSAL.values, -ds.PRES.values, 'r.', label='Salinity')
ax1.set_ylim([-200, 0]); ax1.set_xlim([27, 31]);
ax2.set_ylim([-200, 0]); ax2.set_xlim([35.5, 37.]);

ax1.grid(True)

# Add legends to both axes
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Add axis labels and title
ax1.set_xlabel('Temperature (°C)', color='blue')
ax1.set_ylabel('Pressure (dbar)')
ax2.set_xlabel('Salinity (PSU)', color='red')
#plt.title('Temperature and Salinity Profile')

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(ds.LONGITUDE.values, ds.LATITUDE.values, s=10, label='Argo Profiles')
plt.plot(track.lon, track.lat, 'r-', linewidth=2, label='Hurricane Track')

# Add text labels for the day of each Argo profile
for i in range(len(ds.TIME.values)):
    date_str = pd.to_datetime(ds.TIME.values[i]).strftime('%d')
    plt.text(ds.LONGITUDE.values[i], ds.LATITUDE.values[i], date_str, fontsize=12)

# Add text labels for the start and end of the track
plt.text(track.lon[0].values, track.lat[0].values, pd.to_datetime(track.time[0].values).strftime('%Y-%m-%d'), color='red', fontsize=10)
plt.text(track.lon[-1].values, track.lat[-1].values, pd.to_datetime(track.time[-1].values).strftime('%Y-%m-%d'), color='red', fontsize=10)


plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title(f'{storm} track and Argo Profile Locations')
plt.legend(loc=2)
plt.grid(True)
plt.show()

In [ ]:
ds

In [ ]:
def subset_argo_data(ds, target_date, lon_min, lon_max, lat_min, lat_max):
  """
  Subsets the Argo dataset based on date and geographical range.

  Args:
    ds: The input xarray Dataset containing Argo data.
    target_date: The date to filter by (datetime object or string).
    lon_min: Minimum longitude.
    lon_max: Maximum longitude.
    lat_min: Minimum latitude.
    lat_max: Maximum latitude.

  Returns:
    An xarray Dataset containing the subset of data.
  """
  # Ensure target_date is a datetime object
  if isinstance(target_date, str):
      target_date = pd.to_datetime(target_date)

  # Filter by date (profiles on the target date)
  ds_subset = ds.sel(N_POINTS=(ds.TIME.dt.date == target_date.date()))

  # Filter by geographical range
  ds_subset = ds_subset.where(
      (ds_subset.LONGITUDE >= lon_min) & (ds_subset.LONGITUDE <= lon_max) &
      (ds_subset.LATITUDE >= lat_min) & (ds_subset.LATITUDE <= lat_max),
      drop=True
  )

  return ds_subset

In [ ]:
target_date = '2024-10-08'
lon_min = -87.5
lon_max = -82.5
lat_min = 24
lat_max = 26

subset_ds = subset_argo_data(ds, target_date, lon_min, lon_max, lat_min, lat_max)
display(subset_ds)

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(1, 2, 1)
ax.plot(track.lon, track.lat, 'r-', linewidth=1, label='Hurricane Track')
ax.scatter(subset_ds.LONGITUDE.values, subset_ds.LATITUDE.values, s=10, label='Subset Argo Profiles')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
# Add text labels for the start and end of the track
ax.text(track.lon[0].values, track.lat[0].values, pd.to_datetime(track.time[0].values).strftime('%Y-%m-%d'), color='red', fontsize=10)
ax.text(track.lon[-1].values, track.lat[-1].values, pd.to_datetime(track.time[-1].values).strftime('%Y-%m-%d'), color='red', fontsize=10)
# add date of the profile
for i in range(len(subset_ds.TIME.values)):
    date_str = pd.to_datetime(subset_ds.TIME.values[i]).strftime('%Y-%m-%d')
    ax.text(subset_ds.LONGITUDE.values[i], subset_ds.LATITUDE.values[i], date_str, fontsize=12)

ax1 = fig.add_subplot(1, 2, 2)
ax2 = ax1.twiny()
ax1.plot(subset_ds.TEMP.values, -subset_ds.PRES.values, 'b.', ls='-', label='Temperature')
ax2.plot(subset_ds.PSAL.values, -subset_ds.PRES.values, 'r.', ls= '-', label='Salinity')
ax1.set_ylim([-200, 0]); ax1.set_xlim([27, 31]);
ax2.set_ylim([-200, 0]); ax2.set_xlim([35.5, 37.]);
ax1.grid(True)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.set_xlabel('Temperature (°C)', color='blue')
ax1.set_ylabel('Pressure (dbar)')
ax2.set_xlabel('Salinity (PSU)', color='red')

# Change the color of the y-axis ticks for the temperature axis (ax1) to blue
ax1.tick_params(axis='x', colors='blue')
ax2.tick_params(axis='x', colors='red')